In [ ]:
#pint out versions of sklearn 
import sklearn
import pandas as pd
print(sklearn.__version__) # 1.3.1
print(pd.__version__) # 2.2.2

In [ ]:
# predictive analyses
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import os
import pickle
import seaborn as sns

sns.set(font_scale=1.0)
sns.set_style("ticks")

# Create directory for saving the best models
os.makedirs("best_models", exist_ok=True)

from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

def run_ML(df_dsp_back_ho_norm_new, df_dsp_back_ba_norm_new, genes_of_interest, scatter_size):
    # Prepare cross-validation, plot setup, and other initializations remain the same...
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    models = [Ridge(), RandomForestRegressor(), SVR()]
    params = [
        {
            "alpha": [1e-08, 1e-07, 1e-6, 1e-5, 1e-4, 1e-3, 0.001, 0.01, 0.1, 1, 10, 100]
        },

        # random forest parameters
        {
        # Parameters for Random Forest
        "n_estimators": [100, 300, 500, 1000],
        "max_depth": [None, 3, 5, 7, 9],  # Including None as an option
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
        "bootstrap": [True, False]
        },
        
        {
            "C": [0.001, 0.01, 0.1, 1, 10, 100],
            "kernel": ["linear", "rbf"],
            "gamma": [0.01, 0.1, 1, 10],
            "epsilon": [0.1, 0.2, 0.3, 0.4, 0.5]
        },

    ]
    model_names = ["Ridge Regression", "Random Forest", "SVM"]

    # For each model
    for i in range(len(models)):
        model = models[i]
        param = params[i]
        model_name = model_names[i]
        mses = []
        maes = []
        r2s = []
        evss = []

        model_path = f'best_models_013024/{model_name.replace(" ", "_").lower()}_{genes_of_interest.replace("/", "_")}.pkl'

        if os.path.exists(model_path):  # If the model already exists, load it
            with open(model_path, "rb") as f:
                best_model = pickle.load(f)
            print("Read saved models")
        else:  # Otherwise, perform grid search to find the best model
            grid = GridSearchCV(
                model, param, cv=kfold, scoring="neg_mean_absolute_error", n_jobs=-1
            )
            grid.fit(df_dsp_back_ho_norm_new, df_dsp_back_ba_norm_new)
            best_model = grid.best_estimator_

            # Save the best model
            with open(model_path, "wb") as f:
                pickle.dump(best_model, f)

        residuals = []
        preds = []  # List to store all predictions

        feature_importances = [] 
        # For each split
        for j, (train, test) in enumerate(kfold.split(df_dsp_back_ho_norm_new)):
            X_train, X_test = (
                df_dsp_back_ho_norm_new.iloc[train],
                df_dsp_back_ho_norm_new.iloc[test],
            )
            y_train, y_test = (
                df_dsp_back_ba_norm_new.iloc[train],
                df_dsp_back_ba_norm_new.iloc[test],
            )

            # Fit and predict
            best_model.fit(X_train, y_train)
            pred = best_model.predict(X_test)
            preds.extend(pred)  # Add predictions to the list
            
            # Save feature importances for Random Forest model
            if model_name == "Ridge Regression":
                feature_importances.append(best_model.coef_ )

            mse = mean_squared_error(y_test, pred)
            mses.append(mse)

            mae = mean_absolute_error(y_test, pred)
            maes.append(mae)

            r2 = r2_score(y_test, pred)
            r2s.append(r2)

            evs = explained_variance_score(y_test, pred)
            evss.append(evs)

            residuals.extend(
                y_test - pred
            )  # Calculate residuals and add them to the list

            axs[i].scatter(
                y_test,
                pred,
                alpha=0.6,
                label="Fold {}; MAE: {:.2f}".format(j + 1, mae),
                s=scatter_size,
            )
            
        axs[i].set_title(
            "{} - Avg MAE: {:.3f}, Avg EVS: {:.3f}, Avg R2: {: 3f}".format(
                model_name, np.mean(maes), np.mean(evss), np.mean(r2s)
            )
        )
        print("R2: ", np.mean(r2s))

        axs[i].plot(
            [df_dsp_back_ba_norm_new.min(), df_dsp_back_ba_norm_new.max()],
            [df_dsp_back_ba_norm_new.min(), df_dsp_back_ba_norm_new.max()],
            "k--",
        )
        axs[i].set_xlabel("True Values (" + genes_of_interest + ")", size = 16)
        axs[i].set_ylabel("Predictions (" + genes_of_interest + ")", size = 16)
        axs[i].legend()
        
        if model_name == "Ridge Regression":
            # Calculate average feature importance and rank the features
            avg_feat_imp = np.mean(feature_importances, axis=0)
            ranked_features = np.argsort(avg_feat_imp)[::-1]

            # Save feature importances to a DataFrame
            feat_imp_df = pd.DataFrame({
                "Feature": df_dsp_back_ho_norm_new.columns,
                "Importance": avg_feat_imp
            })
            feat_imp_df['Importance'] = np.abs(feat_imp_df['Importance'])
            feat_imp_df.to_csv(f"feat_imp/{model_name.replace(' ', '_').lower()}_{genes_of_interest.replace('/', '_')}_feature_importances.csv", index=False)

            # Select top 30 important features
            top_30_features = df_dsp_back_ho_norm_new.columns[ranked_features[:100]]

            # Retrain the model using top 30 important features
            best_model.fit(df_dsp_back_ho_norm_new[top_30_features], df_dsp_back_ba_norm_new)

            # Evaluate the model using top 30 important features
            scores = -1 * cross_val_score(best_model, df_dsp_back_ho_norm_new[top_30_features], df_dsp_back_ba_norm_new, 
                                          cv=kfold, scoring="neg_mean_absolute_error", n_jobs=-1)
            print(f"Average MAE for {model_name} with top 100 features: {np.mean(scores):.3f}")

    plt.tight_layout()
    plt.show()
    
for genes_of_interest in ["rplL"]:

    selected_rois = df_dsp_back_ba.loc[genes_of_interest][
        df_dsp_back_ba.loc[genes_of_interest] >= 0
    ].index.tolist()
    
    selected_rois = list(set(meta[meta.Group.isin(['WT_PA'])].index.tolist()) & set(selected_rois))
    
    # df_dsp_back_ba_norm_new = df_dsp_back_ba[selected_rois].loc[genes_of_interest]
    df_dsp_back_ba_norm_new = np.log2(df_dsp_back_ba[selected_rois].loc[genes_of_interest] + 1)
    
    print("bac_shape:", df_dsp_back_ba_norm_new.shape)

    # normalized "df_dsp_back_ho_select" using StandardScaler >> df_dsp_back_ho_norm_new (Host gene expression)
    # df_dsp_back_ho_norm_new's shape is (43, 19962), sharing same columns with "df_dsp_back_ba_norm_new"
    scaler = StandardScaler()
    df_dsp_back_ho_select = df_dsp_back_ho[df_dsp_back_ba_norm_new.index]

    df_dsp_back_ho_norm_new = scaler.fit_transform(df_dsp_back_ho_select.T)
    df_dsp_back_ho_norm_new = pd.DataFrame(
        df_dsp_back_ho_norm_new,
        index=df_dsp_back_ho_select.columns,
        columns=df_dsp_back_ho_select.index,
    )
    print("ho_shape:", df_dsp_back_ho_norm_new.shape)

    run_ML(df_dsp_back_ho_norm_new, df_dsp_back_ba_norm_new, genes_of_interest, scatter_size = 30)

In [ ]:
sns.set(font_scale=1.1)
sns.set_style("ticks")

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

random_state_seed = 40
def evaluate_features_performance(df_dsp_back_ho_norm_new, df_dsp_back_ba_norm_new, top_n_features_list, genes_of_interest):
    kfold = KFold(n_splits=5, shuffle=True, random_state=random_state_seed)
    model_name = "Ridge Regression"
    best_r2_score = -np.inf
    best_n_features = None
    r2_scores_list = []

    # Hyperparameters for Ridge Regression
    params = {
        "alpha": [1e-08, 1e-07, 1e-6, 1e-5, 1e-4, 1e-3, 0.001, 0.01, 0.1, 1, 10, 100]
    }

    for n in top_n_features_list:
        file_path = f"feat_imp/{model_name.replace(' ', '_').lower()}_{genes_of_interest.replace('/', '_')}_feature_importances.csv"
        feat_imp_df = pd.read_csv(file_path)
        feat_imp_df['Importance'] = np.abs(feat_imp_df['Importance'])
        top_features = feat_imp_df.sort_values(by='Importance', ascending=False).head(n)['Feature'].values

        r2_scores = []

        # Create Ridge model and GridSearchCV
        ridge = Ridge()
        grid = GridSearchCV(ridge, params, cv=kfold, scoring="neg_mean_absolute_error", n_jobs=-1)
        grid.fit(df_dsp_back_ho_norm_new[top_features], df_dsp_back_ba_norm_new)
        best_model = grid.best_estimator_

        for train_idx, test_idx in kfold.split(df_dsp_back_ho_norm_new):
            X_train, X_test = df_dsp_back_ho_norm_new.iloc[train_idx][top_features], df_dsp_back_ho_norm_new.iloc[test_idx][top_features]
            y_train, y_test = df_dsp_back_ba_norm_new.iloc[train_idx], df_dsp_back_ba_norm_new.iloc[test_idx]

            best_model.fit(X_train, y_train)

            pred = best_model.predict(X_test)
            r2_scores.append(r2_score(y_test, pred))

        # print best model's parameters
        # print(f"Best model's parameters: {best_model.get_params()}; " + f"R²: {np.mean(r2_scores):.3f}; Top {n} features")

        avg_r2_score = np.mean(r2_scores)
        r2_scores_list.append(avg_r2_score)
        if avg_r2_score > best_r2_score:
            best_r2_score = avg_r2_score
            best_n_features = n

    return best_n_features, r2_scores_list

def run_ML_top_features(df_dsp_back_ho_norm_new, df_dsp_back_ba_norm_new, genes_of_interest, scatter_size, top_n_features, axs):
    kfold = KFold(n_splits=5, shuffle=True, random_state=random_state_seed)

    models = [Ridge()]
    model_names = ["Ridge Regression"]

    for i in range(len(models)):
        model_name = model_names[i]

        # Define hyperparameters for Ridge Regression
        params = {
            "alpha": [1e-08, 1e-07, 1e-6, 1e-5, 1e-4, 1e-3, 0.001, 0.01, 0.1, 1, 10, 100],
        }

        # Grid search to find best parameters
        ridge = Ridge()
        grid = GridSearchCV(ridge, params, cv=kfold, scoring="neg_mean_absolute_error", n_jobs=-1)
        grid.fit(df_dsp_back_ho_norm_new[top_n_features], df_dsp_back_ba_norm_new)
        best_model = grid.best_estimator_

        # best_model = pickle.load(open(f'best_models_013024/{model_name.replace(" ", "_").lower()}_{genes_of_interest.replace("/", "_")}.pkl', 'rb'))
        
        scores = []
        r2_scores = []
        ev_scores = []

        for j, (train, test) in enumerate(kfold.split(df_dsp_back_ho_norm_new[top_n_features])):
            X_train, X_test = (
                df_dsp_back_ho_norm_new[top_n_features].iloc[train],
                df_dsp_back_ho_norm_new[top_n_features].iloc[test],
            )
            y_train, y_test = (
                df_dsp_back_ba_norm_new.iloc[train],
                df_dsp_back_ba_norm_new.iloc[test],
            )

            best_model.fit(X_train, y_train)
            # print best model's parameters
            pred = best_model.predict(X_test)

            mae = mean_absolute_error(y_test, pred)
            scores.append(mae)
            r2_scores.append(r2_score(y_test, pred))
            ev_scores.append(explained_variance_score(y_test, pred))

            axs.scatter(
                y_test,
                pred,
                alpha=0.6,
                label="ROI_Fold {}; MAE: {:.2f}".format(j + 1, mae),
                s=scatter_size,
            )
        
        print(np.mean(r2_scores))

        axs.set_title(
            "{} - Avg MAE: {:.3f}, Avg EVS: {:.3f}; R²: {:.3f}; Top {} feat".format(
                'RR', np.mean(scores), np.mean(ev_scores), np.mean(r2_scores), len(top_n_features)
            )
        )
        axs.set_xlabel("True Values (" + genes_of_interest + ")", size=16)
        axs.set_ylabel("Predictions (" + genes_of_interest + ")", size=16)
        axs.legend()
        axs.plot(
            [df_dsp_back_ba_norm_new.min(), df_dsp_back_ba_norm_new.max()],
            [df_dsp_back_ba_norm_new.min(), df_dsp_back_ba_norm_new.max()],
            "k--",
        )

n_rows = 1
n_cols = 1

genes = ["rplL"]
n_features_list = [10, 20, 30, 80, 100, 150, 200, 300, 400, 500, 600, 700, 800, 900, 1000,
                   2000, 3000, 4000, 5000, 7500, 10000, 11000, 12000, 15000, 19962]  # Adjust as needed

selected_rois = df_dsp_back_ba.loc['rplL'][
        df_dsp_back_ba.loc['rplL'] >= 0
    ].index.tolist()
    
selected_rois = list(set(meta[meta.Group.isin(['WT_PA'])].index.tolist()) & set(selected_rois))

# df_dsp_back_ba_norm_new = df_dsp_back_ba[selected_rois].loc[genes_of_interest]
df_dsp_back_ba_norm_new = np.log2(df_dsp_back_ba[selected_rois].loc['rplL'] + 1)

print("bac_shape:", df_dsp_back_ba_norm_new.shape)

# normalized "df_dsp_back_ho_select" using StandardScaler >> df_dsp_back_ho_norm_new (Host gene expression)
# df_dsp_back_ho_norm_new's shape is (43, 19962), sharing same columns with "df_dsp_back_ba_norm_new"
scaler = StandardScaler()
df_dsp_back_ho_select = df_dsp_back_ho[df_dsp_back_ba_norm_new.index]

df_dsp_back_ho_norm_new = scaler.fit_transform(df_dsp_back_ho_select.T)
df_dsp_back_ho_norm_new = pd.DataFrame(
    df_dsp_back_ho_norm_new,
    index=df_dsp_back_ho_select.columns,
    columns=df_dsp_back_ho_select.index,
)

best_n, r2_scores = evaluate_features_performance(df_dsp_back_ho_norm_new, df_dsp_back_ba_norm_new, n_features_list, 'rplL')

# Find the number of features where R2 first exceeds 0.8
above_08_indices = [i for i, r2 in enumerate(r2_scores) if r2 > 0.8]
if above_08_indices:
    index_first_above_08 = above_08_indices[0]
    n_features_first_above_08 = n_features_list[index_first_above_08]
    r2_first_above_08 = r2_scores[index_first_above_08]
else:
    index_first_above_08 = None
    n_features_first_above_08 = None
    r2_first_above_08 = None

# Find the number of features with the highest R2 score
max_r2 = max(r2_scores)
index_max_r2 = r2_scores.index(max_r2)
best_n_features = n_features_list[index_max_r2]

# Plotting R2 scores vs Number of features
plt.figure(figsize=(10, 4))
plt.plot(n_features_list, r2_scores, marker='x', label='R² Score', alpha=0.6)

# Highlight the maximum point
plt.scatter(best_n_features, max_r2, color='red', s=50, marker='o',
             label=f'Max R²={max_r2:.3f} with {best_n_features} features')

# Highlight the first point where R2 is above 0.8
if index_first_above_08 is not None:
    plt.scatter(n_features_first_above_08, r2_first_above_08, color='blue', s=50, marker='o',
                label=f'First R²>0.8 at {n_features_first_above_08} features')

plt.xlabel('Number of Features (Mouse Genes)')
plt.ylabel('Average R² Score')
plt.title('R² Score vs Number of Mouse Genes')
plt.legend()
plt.grid(True)
plt.show()

ranked_features_all = []

fig, axs = plt.subplots(n_rows, n_cols, figsize=(5, 5))

for idx, genes_of_interest in enumerate(tqdm(genes)):
    selected_rois = df_dsp_back_ba.loc[genes_of_interest][
        df_dsp_back_ba.loc[genes_of_interest] >= 0
    ].index.tolist()
    selected_rois = list(set(meta[meta.Group.isin(['WT_PA'])].index.tolist()) & set(selected_rois))
    
    # df_dsp_back_ba_norm_new = df_dsp_back_ba[selected_rois].loc[genes_of_interest]
    df_dsp_back_ba_norm_new = np.log2(df_dsp_back_ba[selected_rois].loc[genes_of_interest] + 1)
    print("bac_shape:", df_dsp_back_ba_norm_new.shape)

    # normalized "df_dsp_back_ho_select" using StandardScaler >> df_dsp_back_ho_norm_new (Host gene expression)
    # df_dsp_back_ho_norm_new's shape is (43, 19962), sharing same columns with "df_dsp_back_ba_norm_new"
    scaler = StandardScaler()
    df_dsp_back_ho_select = df_dsp_back_ho[df_dsp_back_ba_norm_new.index]
    
    df_dsp_back_ho_norm_new = scaler.fit_transform(df_dsp_back_ho_select.T)
    df_dsp_back_ho_norm_new = pd.DataFrame(
        df_dsp_back_ho_norm_new,
        index=df_dsp_back_ho_select.columns,
        columns=df_dsp_back_ho_select.index,
    )
    print("ho_shape:", df_dsp_back_ho_norm_new.shape)
    
    rf_model_name = 'Ridge Regression'
    avg_feat_imp_file = f"feat_imp/{rf_model_name.replace(' ', '_').lower()}_{genes_of_interest.replace('/', '_')}_feature_importances.csv"
    feat_imp_df = pd.read_csv(avg_feat_imp_file)
    feat_imp_df['Importance'] = np.abs(feat_imp_df['Importance'])
    ranked_features = feat_imp_df.sort_values(by='Importance', ascending=False).head(best_n)['Feature'].values
    ranked_features_all.append(ranked_features)
    run_ML_top_features(df_dsp_back_ho_norm_new, df_dsp_back_ba_norm_new, genes_of_interest, scatter_size=40, top_n_features=ranked_features, axs=axs)

plt.tight_layout()
plt.show()

In [ ]:
from scipy.stats import spearmanr
from statsmodels.stats.multitest import multipletests

meta_wt = meta[meta.Group.str.contains('WT_PA')]

# Create the folder if it doesn't exist
if not os.path.exists("correlation_norm_013124"):
    os.makedirs("correlation_norm_013124")

correlation_file = "correlation_norm_013124/correlation_matrix_wt.csv"
p_value_file = "correlation_norm_013124/p_value_matrix_wt.csv"
fdr_p_value_file = "correlation_norm_013124/fdr_p_value_matrix_wt.csv"

threshold = 0  
if os.path.exists(correlation_file) and os.path.exists(p_value_file):
    
    # Normalize expression values (e.g., log transformation)
    df_dsp_back_ho_norm = df_dsp_back_ho.apply(lambda x: np.log2(x + 1))[meta_wt[meta_wt.index.isin(set(df_dsp_back_ba.columns) & set(df_dsp_back_ho.columns))].index]
    df_dsp_back_ba_norm = df_dsp_back_ba.apply(lambda x: np.log2(x + 1))[meta_wt[meta_wt.index.isin(set(df_dsp_back_ba.columns) & set(df_dsp_back_ho.columns))].index]
    
    # Filter out lowly expressed genes (optional)
    df_dsp_back_ho_norm = df_dsp_back_ho_norm[df_dsp_back_ho_norm.apply(lambda x: x >= threshold).any(axis=1)]
    df_dsp_back_ba_norm = df_dsp_back_ba_norm[df_dsp_back_ba_norm.apply(lambda x: x >= threshold).any(axis=1)]
    
    print(df_dsp_back_ho_norm.shape, df_dsp_back_ba_norm.shape)
    
    # Load the matrices from the saved files
    correlation_matrix = np.loadtxt(correlation_file, delimiter=",")
    p_value_matrix = np.loadtxt(p_value_file, delimiter=",")
    
    # Handle potential nan values in p_value_matrix
    p_value_matrix = np.nan_to_num(p_value_matrix, nan=1.0)
    
    # Apply FDR correction
    p_value_matrix = p_value_matrix.flatten()
    _, fdr_corrected_p_values, _, _ = multipletests(p_value_matrix, method='fdr_bh')
    fdr_corrected_p_values = fdr_corrected_p_values.reshape(correlation_matrix.shape)
    
    # Save the FDR corrected p-values
    np.savetxt(fdr_p_value_file, fdr_corrected_p_values, delimiter=",")
else:
    # Normalize expression values (e.g., log transformation)
    df_dsp_back_ho_norm = df_dsp_back_ho.apply(lambda x: np.log2(x + 1))[meta_wt[meta_wt.index.isin(set(df_dsp_back_ba.columns) & set(df_dsp_back_ho.columns))].index]
    df_dsp_back_ba_norm = df_dsp_back_ba.apply(lambda x: np.log2(x + 1))[meta_wt[meta_wt.index.isin(set(df_dsp_back_ba.columns) & set(df_dsp_back_ho.columns))].index]

    # Filter out lowly expressed genes (optional)
    df_dsp_back_ho_norm = df_dsp_back_ho_norm[df_dsp_back_ho_norm.apply(lambda x: x >= threshold).any(axis=1)]
    df_dsp_back_ba_norm = df_dsp_back_ba_norm[df_dsp_back_ba_norm.apply(lambda x: x >= threshold).any(axis=1)]

    print(df_dsp_back_ho_norm.shape, df_dsp_back_ba_norm.shape)

    # Calculate correlation and p-value matrices
    num_host_genes = df_dsp_back_ho_norm.shape[0]
    num_bacterial_genes = df_dsp_back_ba_norm.shape[0]
    correlation_matrix = np.zeros((num_host_genes, num_bacterial_genes))
    p_value_matrix = np.zeros((num_host_genes, num_bacterial_genes))

    for i, host_gene in enumerate(df_dsp_back_ho_norm.index):
        for j, bacterial_gene in enumerate(df_dsp_back_ba_norm.index):
            correlation_matrix[i, j], p_value_matrix[i, j] = spearmanr(df_dsp_back_ho_norm.loc[host_gene], df_dsp_back_ba_norm.loc[bacterial_gene])

    # Handle potential nan values in p_value_matrix
    p_value_matrix = np.nan_to_num(p_value_matrix, nan=1.0)
    
    # Apply FDR correction
    p_value_matrix = p_value_matrix.flatten()
    _, fdr_corrected_p_values, _, _ = multipletests(p_value_matrix, method='fdr_bh')
    fdr_corrected_p_values = fdr_corrected_p_values.reshape(correlation_matrix.shape)

    # Save the matrices to the folder
    np.savetxt(correlation_file, correlation_matrix, delimiter=",")
    np.savetxt(p_value_file, p_value_matrix, delimiter=",")
    np.savetxt(fdr_p_value_file, fdr_corrected_p_values, delimiter=",")

In [ ]:
# Set thresholds
correlation_threshold = 0.7  # Adjust this value based on your desired correlation threshold
alpha = 0.05  # Adjust this value based on your desired significance level

# Identify significant interactions
significant_interactions = np.where((np.abs(correlation_matrix) > correlation_threshold) & (fdr_corrected_p_values < alpha))

import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
from adjustText import adjust_text

sns.set(font_scale=1.4)
sns.set_style("ticks")

# Create interaction network graph
fig, ax = plt.subplots(figsize=(7, 7))
interaction_network = nx.Graph()

##########human ppi to add
corr_host_genes = pd.DataFrame([x for x in list(interaction_network.nodes()) if x != 'rplL'])
corr_host_genes.to_csv('correlation_norm_013124/corr_host_genes.txt', index=False, header=False)

# add ppi between host genes
df = pd.read_excel('correlation_norm_013124/PPIs_corr_host/PPIData_1.xlsx')
df2 = pd.read_excel('correlation_norm_013124/PPIs_corr_host/PPIData_2.xlsx')
df3 = pd.read_excel('correlation_norm_013124/PPIs_corr_host/PPIData_3.xlsx')
# combined
df = df[df['Entrez GeneID A'].notnull()]
df2 = df2[df2['Entrez GeneID A'].notnull()]
df3 = df3[df3['Entrez GeneID A'].notnull()]

df_human_ppi = pd.concat([df, df2, df3])
df_human_ppi = df_human_ppi[df_human_ppi['Entrez GeneID A'] != df_human_ppi['Entrez GeneID B']]

# change to lower case for gene names except for the first letter
df_human_ppi['Symbol A'] = df_human_ppi['Symbol A'].apply(lambda x: x[0] + x[1:].lower())
df_human_ppi['Symbol B'] = df_human_ppi['Symbol B'].apply(lambda x: x[0] + x[1:].lower())

# Loop through the human-human PPI DataFrame to add each interaction
for index, row in df_human_ppi.iterrows():
    gene_a = row['Symbol A']
    gene_b = row['Symbol B']
    # Add an edge for each human-human PPI
    interaction_network.add_edge(gene_a, gene_b, type='host_host')

###################

# Define host_genes and bacterial_genes
host_genes = df_dsp_back_ho_norm.index
bacterial_genes = df_dsp_back_ba_norm.index

# Find the index for 'rplL' in bacterial_genes
rplL_index = np.where(bacterial_genes == 'rplL')[0][0]  # Assuming 'rplL' exists in bacterial_genes

# Extract correlations and FDR values for interactions between 'Pmaip1', 'Tnf', 'Trim13' and 'rplL'
target_genes = ['Pmaip1', 'Tnf', 'Trim13']
for gene in target_genes:
    if gene in host_genes:
        gene_idx = np.where(host_genes == gene)[0][0]  # Get the host gene index
        if (gene_idx, rplL_index) in zip(*significant_interactions):
            rho_value = correlation_matrix[gene_idx, rplL_index]
            fdr_value = fdr_corrected_p_values[gene_idx, rplL_index]
            print(f"Interaction between {gene} and rplL: rho = {rho_value:.3f}, FDR = {fdr_value:.2e}")

# Add edges and nodes with links
linked_host_genes = set()
linked_bacterial_genes = set()

for i, j in zip(*significant_interactions):
    host_gene = host_genes[i]
    bacterial_gene = bacterial_genes[j]
    if bacterial_gene == 'rplL':
        # weight = correlation_matrix[i, j]
        interaction_network.add_edge(host_gene, bacterial_gene, type='host_rplL')
        linked_host_genes.add(host_gene)
        linked_bacterial_genes.add(bacterial_gene)

# Create a dictionary to map each linked bacterial gene to a unique color
linked_bacterial_gene_colors = {bacterial_gene: plt.cm.get_cmap('tab10')(i % 20) for i, bacterial_gene in enumerate(linked_bacterial_genes)}

# Set edge colors
edge_colors = []
for u, v, data in interaction_network.edges(data=True):
    if data.get('type') == 'host_host':
        edge_colors.append('orange')  # Human-human PPIs in red
    elif data.get('type') == 'host_rplL':
        # Retain original coloring logic for other interactions
        edge_colors.append('steelblue')
# Calculate node degrees
node_degrees = dict(interaction_network.degree())

# Set node sizes based on degree
# node_sizes = [10 * node_degrees[node] for node in interaction_network.nodes()]
node_sizes = [30  for node in interaction_network.nodes()]

# Set node colors
node_colors = ["skyblue" if n in linked_host_genes else "lightcoral" for n in interaction_network.nodes()]

# Custom layout function to evenly distribute nodes with equal spacing
def equal_spacing_layout(G, linked_bacterial_genes):
    pos = nx.bipartite_layout(G, linked_bacterial_genes)

    # Calculate the number of host and bacterial nodes
    num_host_nodes = len(G.nodes) - len(linked_bacterial_genes)
    num_bacterial_nodes = len(linked_bacterial_genes)

    # Assign equal spacing for bacterial nodes
    bacterial_spacing = 1 / (num_bacterial_nodes + 1)
    for i, node in enumerate(sorted(linked_bacterial_genes, key=lambda x: G.degree(x), reverse=False)):
        x, _ = pos[node]
        pos[node] = (x, (i + 1) * bacterial_spacing)

    # Assign equal spacing for host nodes
    host_spacing = 1 / (num_host_nodes + 1)
    for i, node in enumerate(sorted(G.nodes - linked_bacterial_genes, key=lambda x: G.degree(x), reverse=False)):
        x, _ = pos[node]
        pos[node] = (x, (i + 1) * host_spacing)

    return pos

# Visualize the network
pos = nx.kamada_kawai_layout(interaction_network, scale=1)
nx.draw(interaction_network, pos, node_color=node_colors, edge_color=edge_colors, width=2, node_size=node_sizes, alpha= 0.4)

# Add labels with adjusted positions to prevent overlap
texts = []
for node, (x, y) in pos.items():
    if node == 'rplL':  # Only label 'rplL'
        texts.append(plt.text(x, y, node, ha='center', va='center', fontsize=16))
    elif node in linked_host_genes:
        texts.append(plt.text(x, y, node, ha='center', va='center', fontsize=12))

adjust_text(texts, arrowprops=dict(arrowstyle="-", color='grey', lw=0.5))

plt.title('Host-Bacteria Gene Expression Corr_Net (rho > '  + str(correlation_threshold)  + '; FDR < ' + str(alpha) + ')', fontsize = 13)
plt.show()

In [ ]:
# different folds represent different biological replicates
meta_wt = meta[meta.Group.str.contains('WT_PA')]
meta_wt.loc[ meta_wt['Scan name'].str.startswith('A'), 'mouse_id'] = 'A'
meta_wt.loc[ meta_wt['Scan name'].str.startswith('B'), 'mouse_id'] = 'B'

from sklearn.model_selection import GroupKFold

sns.set(font_scale=1.1)
sns.set_style("ticks")

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

def evaluate_features_performance(df_dsp_back_ho_norm_new, df_dsp_back_ba_norm_new, top_n_features_list, genes_of_interest, meta_wt):

    # kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    model_name = "Ridge Regression"
    best_r2_score = -np.inf
    best_n_features = None
    r2_scores_list = []

    # Hyperparameters for Ridge Regression
    params = {
        "alpha": [1e-08, 1e-07, 1e-6, 1e-5, 1e-4, 1e-3, 0.001, 0.01, 0.1, 1, 10, 100]
    }

    gkf = GroupKFold(n_splits=2)
    groups = meta_wt.loc[df_dsp_back_ho_norm_new.index, 'mouse_id']

    for n in top_n_features_list:
        file_path = f"feat_imp/{model_name.replace(' ', '_').lower()}_{genes_of_interest.replace('/', '_')}_feature_importances.csv"
        feat_imp_df = pd.read_csv(file_path)
        feat_imp_df['Importance'] = np.abs(feat_imp_df['Importance'])
        top_features = feat_imp_df.sort_values(by='Importance', ascending=False).head(n)['Feature'].values

        r2_scores = []

        # Create Ridge model and GridSearchCV
        ridge = Ridge()
        grid = GridSearchCV(ridge, params, cv=gkf, scoring="neg_mean_absolute_error",  n_jobs=-1)
        grid.fit(df_dsp_back_ho_norm_new[top_features], df_dsp_back_ba_norm_new, groups=groups,)
        best_model = grid.best_estimator_

        for train_idx, test_idx in gkf.split(df_dsp_back_ho_norm_new, groups=groups):
            X_train, X_test = df_dsp_back_ho_norm_new.iloc[train_idx][top_features], df_dsp_back_ho_norm_new.iloc[test_idx][top_features]
            y_train, y_test = df_dsp_back_ba_norm_new.iloc[train_idx], df_dsp_back_ba_norm_new.iloc[test_idx]

            best_model.fit(X_train, y_train)

            pred = best_model.predict(X_test)
            r2_scores.append(r2_score(y_test, pred))

        # print best model's parameters
        # print(f"Best model's parameters: {best_model.get_params()}; " + f"R²: {np.mean(r2_scores):.3f}; Top {n} features")

        avg_r2_score = np.mean(r2_scores)
        r2_scores_list.append(avg_r2_score)
        if avg_r2_score > best_r2_score:
            best_r2_score = avg_r2_score
            best_n_features = n

    return best_n_features, r2_scores_list

def run_ML_top_features(df_dsp_back_ho_norm_new, df_dsp_back_ba_norm_new, genes_of_interest, scatter_size, top_n_features, axs, meta_wt):

    gkf = GroupKFold(n_splits=2)
    groups = meta_wt.loc[df_dsp_back_ho_norm_new.index, 'mouse_id']

    # kfold = KFold(n_splits=2, shuffle=True, random_state=42)

    models = [Ridge()]
    model_names = ["Ridge Regression"]

    for i in range(len(models)):
        model_name = model_names[i]

        # Define hyperparameters for Ridge Regression
        params = {
            "alpha": [1e-08, 1e-07, 1e-6, 1e-5, 1e-4, 1e-3, 0.001, 0.01, 0.1, 1, 10, 100],
        }

        # Grid search to find best parameters
        ridge = Ridge()
        grid = GridSearchCV(ridge, params, cv=gkf, scoring="neg_mean_absolute_error", n_jobs=-1)
        grid.fit(df_dsp_back_ho_norm_new[top_n_features], df_dsp_back_ba_norm_new,groups=groups )
        best_model = grid.best_estimator_

        # best_model = pickle.load(open(f'best_models_013024/{model_name.replace(" ", "_").lower()}_{genes_of_interest.replace("/", "_")}.pkl', 'rb'))
        
        scores = []
        r2_scores = []
        ev_scores = []

        for j, (train, test) in enumerate(gkf.split(df_dsp_back_ho_norm_new[top_n_features], groups=groups)):
            X_train, X_test = (
                df_dsp_back_ho_norm_new[top_n_features].iloc[train],
                df_dsp_back_ho_norm_new[top_n_features].iloc[test],
            )
            y_train, y_test = (
                df_dsp_back_ba_norm_new.iloc[train],
                df_dsp_back_ba_norm_new.iloc[test],
            )

            best_model.fit(X_train, y_train)
            # print best model's parameters
            pred = best_model.predict(X_test)

            mae = mean_absolute_error(y_test, pred)
            scores.append(mae)
            r2_scores.append(r2_score(y_test, pred))
            ev_scores.append(explained_variance_score(y_test, pred))

            axs.scatter(
                y_test,
                pred,
                alpha=0.6,
                label="Host_Fold {}; MAE: {:.2f}".format(j + 1, mae),
                s=scatter_size,
            )
        
        print(np.mean(r2_scores))

        axs.set_title(
            "{} - Avg MAE: {:.3f}, Avg EVS: {:.3f}; R²: {:.3f}; Top {} feat".format(
                'RR', np.mean(scores), np.mean(ev_scores), np.mean(r2_scores), len(top_n_features)
            )
        )
        axs.set_xlabel("True Values (" + genes_of_interest + ")", size=16)
        axs.set_ylabel("Predictions (" + genes_of_interest + ")", size=16)
        axs.legend()
        axs.plot(
            [df_dsp_back_ba_norm_new.min(), df_dsp_back_ba_norm_new.max()],
            [df_dsp_back_ba_norm_new.min(), df_dsp_back_ba_norm_new.max()],
            "k--",
        )

n_rows = 1
n_cols = 1

genes = ["rplL"]
n_features_list = [10, 20, 30, 80, 100, 150, 200, 300, 400, 500, 600, 700, 800, 900, 1000,
                   2000, 3000, 4000, 5000, 7500, 10000, 11000, 12000, 15000, 19962]  # Adjust as needed

selected_rois = df_dsp_back_ba.loc['rplL'][
        df_dsp_back_ba.loc['rplL'] >= 0
    ].index.tolist()
    
selected_rois = list(set(meta[meta.Group.isin(['WT_PA'])].index.tolist()) & set(selected_rois))

# df_dsp_back_ba_norm_new = df_dsp_back_ba[selected_rois].loc[genes_of_interest]
df_dsp_back_ba_norm_new = np.log2(df_dsp_back_ba[selected_rois].loc['rplL'] + 1)

print("bac_shape:", df_dsp_back_ba_norm_new.shape)

# normalized "df_dsp_back_ho_select" using StandardScaler >> df_dsp_back_ho_norm_new (Host gene expression)
# df_dsp_back_ho_norm_new's shape is (43, 19962), sharing same columns with "df_dsp_back_ba_norm_new"
scaler = StandardScaler()
df_dsp_back_ho_select = df_dsp_back_ho[df_dsp_back_ba_norm_new.index]

df_dsp_back_ho_norm_new = scaler.fit_transform(df_dsp_back_ho_select.T)
df_dsp_back_ho_norm_new = pd.DataFrame(
    df_dsp_back_ho_norm_new,
    index=df_dsp_back_ho_select.columns,
    columns=df_dsp_back_ho_select.index,
)

# Evaluate the performance of features
best_n, r2_scores = evaluate_features_performance(df_dsp_back_ho_norm_new, df_dsp_back_ba_norm_new, n_features_list, 'rplL', meta_wt)

# Find the number of features where R2 first exceeds 0.8
above_08_indices = [i for i, r2 in enumerate(r2_scores) if r2 > 0.8]
if above_08_indices:
    index_first_above_08 = above_08_indices[0]
    n_features_first_above_08 = n_features_list[index_first_above_08]
    r2_first_above_08 = r2_scores[index_first_above_08]
else:
    index_first_above_08 = None
    n_features_first_above_08 = None
    r2_first_above_08 = None

# Find the number of features with the highest R2 score
max_r2 = max(r2_scores)
index_max_r2 = r2_scores.index(max_r2)
best_n_features = n_features_list[index_max_r2]

# Plotting R2 scores vs Number of features
plt.figure(figsize=(10, 4))
plt.plot(n_features_list, r2_scores, marker='x', label='R² Score', alpha=0.6)

# Highlight the maximum point
plt.scatter(best_n_features, max_r2, color='red', s=50, marker='o',
             label=f'Max R²={max_r2:.3f} with {best_n_features} features')

# Highlight the first point where R2 is above 0.8
if index_first_above_08 is not None:
    plt.scatter(n_features_first_above_08, r2_first_above_08, color='blue', s=50, marker='o',
                label=f'First R²>0.8 at {n_features_first_above_08} features')

plt.xlabel('Number of Features (Mouse Genes)')
plt.ylabel('Average R² Score')
plt.title('R² Score vs Number of Mouse Genes')
plt.legend()
plt.grid(True)
plt.show()

ranked_features_all = []

fig, axs = plt.subplots(n_rows, n_cols, figsize=(5, 5))

for idx, genes_of_interest in enumerate(tqdm(genes)):
    selected_rois = df_dsp_back_ba.loc[genes_of_interest][
        df_dsp_back_ba.loc[genes_of_interest] >= 0
    ].index.tolist()
    selected_rois = list(set(meta[meta.Group.isin(['WT_PA'])].index.tolist()) & set(selected_rois))
    
    # df_dsp_back_ba_norm_new = df_dsp_back_ba[selected_rois].loc[genes_of_interest]
    df_dsp_back_ba_norm_new = np.log2(df_dsp_back_ba[selected_rois].loc[genes_of_interest] + 1)
    print("bac_shape:", df_dsp_back_ba_norm_new.shape)

    # normalized "df_dsp_back_ho_select" using StandardScaler >> df_dsp_back_ho_norm_new (Host gene expression)
    # df_dsp_back_ho_norm_new's shape is (43, 19962), sharing same columns with "df_dsp_back_ba_norm_new"
    scaler = StandardScaler()
    df_dsp_back_ho_select = df_dsp_back_ho[df_dsp_back_ba_norm_new.index]
    
    df_dsp_back_ho_norm_new = scaler.fit_transform(df_dsp_back_ho_select.T)
    df_dsp_back_ho_norm_new = pd.DataFrame(
        df_dsp_back_ho_norm_new,
        index=df_dsp_back_ho_select.columns,
        columns=df_dsp_back_ho_select.index,
    )
    print("ho_shape:", df_dsp_back_ho_norm_new.shape)
    
    rf_model_name = 'Ridge Regression'
    avg_feat_imp_file = f"feat_imp/{rf_model_name.replace(' ', '_').lower()}_{genes_of_interest.replace('/', '_')}_feature_importances.csv"
    feat_imp_df = pd.read_csv(avg_feat_imp_file)
    feat_imp_df['Importance'] = np.abs(feat_imp_df['Importance'])
    ranked_features = feat_imp_df.sort_values(by='Importance', ascending=False).head(best_n)['Feature'].values
    ranked_features_all.append(ranked_features)
    run_ML_top_features(df_dsp_back_ho_norm_new, df_dsp_back_ba_norm_new, genes_of_interest, 40, ranked_features, axs, meta_wt)

plt.tight_layout()
plt.show()